현진: 호기심
원준: 플래그/폴리시
나: 예측 기반 오차/오류





에피소드 기반 보상은 나중에

In [ ]:
# 에러 기반

def error():

In [ ]:
# 예측기반 오차

# 전체 파라미터를 트리 구조로 바꾸는 코드


def prophecy():
    next_output_value_type = 0 # 여기에 예측 코드 넣으면 됨
    return next_output_value_type

def main(prophecy_out, real_out):
    # 유전 거리 계산 코드
    return

In [ ]:
#뭐가 어떻게 될지 몰라 일단 함수 기틀만 만들어 보았음.
'''우선, 호기심 보상 함수를 두가지 버전으로 만들어보앗는데,
첫번째로는 은세가 말했던 스텝이 진행될 수록 기틀이 잡히니까 호기심 보상을 자동적으로 감쇠시키는 버전,
두번째로는 후반에 가서도 호기심 보상을 감소시키지 않을 수 있게 최악의 상태일떄 음의 보상으로 패치하는 버전으로 만들었음.
'''


#자동 감쇠형 호기심 함수
def curiosity_reward_decay(step, base_reward=1.0, decay_rate=0.005):
    """
    호기심 보상이 스텝(step)에 따라 자연스럽게 감소하는 함수.
    예: 초반엔 높은 보상, 후반으로 갈수록 점점 감소.

    R_c(t) = base_reward * (1 - decay_rate * step)
    (단, 0보다 작으면 0으로 제한)
    """
    reward = max(base_reward * (1 - decay_rate * step), 0)
    return reward
# 최악의 경우에 음의 보상을 주는 호기심 함수
def curiosity_reward_with_penalty(is_redundant=False, is_error=False, step=0,
                                  base_reward=1.0, penalty_redundant=0.3, penalty_error=0.5,
                                  decay_rate=0.003):
    """
    스텝 수에 따른 호기심 보상 감소 + 잘못된 행동에 대한 음의 보상 적용.

    Parameters:
    - is_redundant: 이미 여러 번 시도된 동일한 행동 수행시 True ---
    - is_error: 시스템 오류를 유발하는 탐색시 True >>>
    - step: 현재 스텝 수 >>>
    - base_reward: 기본 호기심 보상 >>>
    - penalty_redundant: 반복 행동에 부여할 음의 보상 강도
    - penalty_error: 심각한 오류 행위에 부여할 음의 보상 강도
    - decay_rate: 스텝에 따른 감쇠 비율
    """
    reward = max(base_reward * (1 - decay_rate * step), 0)
    if is_redundant:
        reward -= penalty_redundant
    if is_error:
        reward -= penalty_error
    return max(reward, -1.0)



# 새로운 행동 -> 보상

# 시간 감쇠

# 연속 반복 페널티


In [ ]:
import hashlib

def flag_reward(flag_str, method, known_flags, found_paths):
    """
    FLAG 보상 함수 (기틀 버전)
    -----------------------------------------------------
    플래그 문자열을 발견했을 때 보상을 계산하는 함수.

    매개변수 설명:
    - flag_str (str): 코드 속에 끼워져 있는 플래그 원문 문자열
    - method (str): 해당 플래그를 찾은 탐색 경로나 행동(명령) 이름
    - known_flags (dict): 사전 등록된 정답 플래그 해시 목록 {flag_name: sha256_hex}
    - found_paths (dict): 이미 찾은 플래그 및 경로 기록 {flag_name: [methods]}
    -----------------------------------------------------
    반환값:
    - reward (float): 계산된 최종 보상값 (단일 스칼라)
    """

    # --- 기본 보상 정책(수정 가능) ---
    BASE_REWARD = 5.0         # 올바른 플래그를 찾았을 때의 기본 보상
    NEW_PATH_BONUS = 2.0      # 같은 플래그를 '새로운 경로'로 찾았을 때 보너스

    reward = 0.0  # 최종 보상 초기화

    # 1) 플래그 해시 계산 (sha256 권장: 실행 간 일관성 확보)
    flag_hash = hashlib.sha256(flag_str.encode("utf-8")).hexdigest()

    # ================================================================
    # [PARSE_HOOK - START]

    #
    # 아래 변수들은 '존재하면 보상 가중에 반영'되도록 설계됨.
    parsed_reward = None   # 예: 문자열에서 읽은 가산 보상(float). 없으면 None 유지
    parsed_flag_id = None  # 예: 플래그를 특정하는 ID가 있다면 사용(str)
    parsed_tags = None     # 예: ['web','sql'] 같은 태그 리스트. 정책에 활용 가능
   
    # ================================================================

    # 2) known_flags와 해시 매칭
    matched_flag_name = None
    for flag_name, valid_hash in known_flags.items():
        if flag_hash == valid_hash:
            matched_flag_name = flag_name
            break

    # 3) 매칭 실패 시 보상 0 반환
    if matched_flag_name is None:
        return 0.0

    # 4) 경로 중복 여부 확인 및 보상 결정
    prior_methods = found_paths.get(matched_flag_name, [])

    if method not in prior_methods:
        # 새로운 경로에서 같은 플래그를 처음 발견
        reward = BASE_REWARD + NEW_PATH_BONUS
        found_paths.setdefault(matched_flag_name, []).append(method)
    else:
        # 동일 경로로 이미 발견한 적 있음 → 기본 보상만
        reward = BASE_REWARD

    # 5) (선택) 파싱된 보상 parsed_reward가 있으면 가산
    #    친구가 파싱 부분에서 parsed_reward를 채우면 자동 반영됨.
    if parsed_reward is not None:
        # 필요 시 클리핑/스케일 조정 가능 (예: max(0, min(parsed_reward, 10)))
        reward += float(parsed_reward)

    # 6) (선택) parsed_flag_id / parsed_tags 활용 훅
    #    여기서 정책적으로 특정 ID나 태그에 가중치를 줄 수도 있음.

    # 7) 최종 스칼라 보상 반환
    return float(reward)


In [ ]:
def main():   
    error_reward()
    prophecy_reward()
    flag_reward()
    curiosity()
    return

In [ ]:
# Policy (ε-탐욕 기반)
def policy_update(actions, Q, epsilon=0.1, alpha=0.1, last_action=None, reward=0):
    """
    간단한 ε-탐욕 정책 함수 (기틀 버전)
    - actions: 가능한 행동 리스트
    - Q: 행동 가치 테이블 (dict)
    - epsilon: 무작위 탐색 확률
    - alpha: 학습률
    - last_action: 직전 수행한 행동
    - reward: 직전 보상
    """
    import random

    # Q값 업데이트 (최근 보상 반영)
    if last_action is not None:
        old_value = Q.get(last_action, 0)
        Q[last_action] = old_value + alpha * (reward - old_value)

    # ε-탐욕 정책으로 다음 행동 선택
    if random.random() < epsilon:
        next_action = random.choice(actions)
    else:
        next_action = max(actions, key=lambda a: Q.get(a, 0))

    return next_action